# Healthy Personalized Food Recommender System Using Two-Tower Deep Neural Network

# Motivation 

Recommender systems are designed to assist users in discovering relevant items by analyzing their preferences and generating a tailored list of suggestions. These systems effectively address the problem of choice overload, enabling users to navigate through vast amount of options with ease. By accurately predicting user preferences, recommender systems enhance user experience and satisfaction. They are widely employed across various domains, offering personalized recommendations that cater to individual needs and interest.
Recommender system employs three different approaches to generate recommendations, collaborative filtering, content based filtering and hybrid approach. 


In the domain of food, recommender systems are utilized in various contexts such as restaurant suggestions, grocery store item personalization, and recipe recommendations. Creating an accurate food recommender system is complex due to the multifaceted nature of the food domain, which involves context, location, health preferences, and more.


## Problem Formulation

Deciding what and where to eat is challenging due to the vast array of available options. Online food recommender systems aim to help users find relevant food items and recipes. However, previous studies show that these systems often promote unhealthy food choices. This is primarily due to user-generated content that favors unhealthy options and the use of datasets that do not comply with health metrics like the Food Standards Agency (FSA) score and World Health Organization (WHO) guidelines.

The goal of this project is to develop a ranking food recommender system that suggests top recipe recommendations. A subsequent healthiness filter will rank these recommendations based on the healthiness of the recipes, using the FSA score. The recommendation generation will be based on the well-known Two-Tower architecture, a popular deep learning approach for building recommender systems.


The proposed model will be evaluated against a baseline: a traditional Singular Value Decomposition (SVD) collaborative filtering model. User recipe ratings will be used to assess the effectiveness of both models.

The future application of this project will involve conducting a user study to provide personalized recommendations using the Two-Tower deep learning model to predict personalized recipes according to each user preference and features. A healthiness filter algorithm will select the healthiest recipes to present to users. The key evaluative question will be:

To what extent does a highly personalized recommender system nudge users toward healthier food choices?

This notebook is organized into the following sections:

- **Dataset:** Overview of the dataset used to train the model
- **Model Architecture:** Details of the Two-Tower deep neural network
- **SVD Baseline:** Description of the baseline collaborative filtering model
- **Evaluation:** Results and evaluations of the developed models

## Dataset

The dataset that we will use in this project is collected during the course line of my PhD project and the user experiments that we condactated as mean to evaluate our propesed recommender systems and digital nudges. The dataset collected from several experiments that can found :

- [ [1] Nudging Towards Health? Examining the Merits of Nutrition
Labels and Personalization in a Recipe Recommender System](https://dl.acm.org/doi/pdf/10.1145/3503252.3531312) 
- [ [2] Boosting Health? Examining the Role of Nutrition
Labels and Preference Elicitation Methods in Food
Recommendation](https://edepot.wur.nl/579903) 
- [ [3] The Interplay between Food Knowledge, Nudges, and
Preference Elicitation Methods Determines the
Evaluation of a Recipe Recommender System](https://ceur-ws.org/Vol-3534/paper1.pdf) 
- [ [4] Nudging towards sustainable recipes: Nudging towards sustainable recipes, Internal project]()


### Data Preparation and EDA

- The dataset is from different projects and the main idea is to construct training data from the previously collected data: user features dataset and a dataframe that describes recipe features.

In [2]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### User Features
Construct a user training dataset that describes each user through various features, derived by combining different dataframes from the above projects.

In [3]:
##  user data from Nudging towards healthy food choice [1]
nudge_pers_df = pd.read_csv('./Dataset/Nudging_personalization.csv')

## user data from Boosting healthy [2]
boosting_heal_df = pd.read_csv('./Dataset/PE_boost_nudge_study.csv')

## user data from Interplay between food knowledge and preference elicitation [3]
preference_user_df = pd.read_csv('./Dataset/PE_elicitation.csv')

## sustainability data
sustainability_user_id = pd.read_csv('./Dataset/Sustainability_data.csv')

- Select useful features from each DF

In [4]:
## DF [1]
nudge_pers_df.info()
nudge_pers_df = nudge_pers_df[['person','age','country','education','diet_restriction','diet_goal', 
                               'cooking_exp','eating_habits','gender','recipe_name']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   person               300 non-null    int64 
 1   age                  300 non-null    object
 2   country              300 non-null    object
 3   education            300 non-null    object
 4   diet_restriction     300 non-null    object
 5   diet_goal            300 non-null    object
 6   cooking_exp          300 non-null    object
 7   eating_habits        300 non-null    object
 8   gender               300 non-null    object
 9   recipe_name          300 non-null    object
 10  healthiness          300 non-null    object
 11  Nutri_score          300 non-null    object
 12  fsa_score            300 non-null    int64 
 13  recommend_recipe     300 non-null    object
 14  become_favorite      300 non-null    object
 15  enjoy_eating         300 non-null    object
 16  many_to_

In [5]:
## DF [2]
boosting_heal_df.info()
boosting_heal_df = boosting_heal_df[['person','age','country','education','gender','recipe_name']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               243 non-null    int64  
 1   age                  243 non-null    object 
 2   country              243 non-null    object 
 3   education            243 non-null    object 
 4   gender               243 non-null    object 
 5   FK_1                 243 non-null    int64  
 6   FK_2                 243 non-null    int64  
 7   FK_3                 243 non-null    int64  
 8   FK_4                 243 non-null    int64  
 9   FK_5                 243 non-null    int64  
 10  category             243 non-null    object 
 11  recipe_id            243 non-null    int64  
 12  recipe_name          243 non-null    object 
 13  Nutri_score          243 non-null    object 
 14  fsa_score            243 non-null    int64  
 15  healthiness          243 non-null    obj

In [6]:
## DF [3] 
preference_user_df.info()
preference_user_df = preference_user_df[['person','age','country','education','gender','recipe_name','Height',
                                         'Weight','RecipeWebUsage','HomeCook','CookingExp',
                                         'EatingGoals','Depression','PhysicalActivity','SleepHours',
                                         'CookingTime']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               360 non-null    int64  
 1   age                  360 non-null    object 
 2   country              360 non-null    object 
 3   education            360 non-null    object 
 4   gender               360 non-null    object 
 5   FK_9                 360 non-null    int64  
 6   FK_10                360 non-null    int64  
 7   FK_11                360 non-null    int64  
 8   FK_12                360 non-null    int64  
 9   Height               180 non-null    float64
 10  Weight               180 non-null    float64
 11  RecipeWebUsage       180 non-null    object 
 12  HomeCook             180 non-null    object 
 13  CookingExp           180 non-null    object 
 14  EatingGoals          180 non-null    object 
 15  Depression           180 non-null    obj

In [7]:
## DF [4] 
sustainability_user_id.info()
sustainability_user_id = sustainability_user_id[['person','age','country','education','gender',
                                 'recipe_name','diet_restriction','diet_goal','cooking_exp','eating_habits']] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               200 non-null    int64  
 1   age                  200 non-null    object 
 2   country              200 non-null    object 
 3   education            200 non-null    object 
 4   diet_restriction     200 non-null    object 
 5   diet_goal            200 non-null    object 
 6   cooking_exp          200 non-null    object 
 7   eating_habits        200 non-null    object 
 8   gender               200 non-null    object 
 9   category             200 non-null    object 
 10  recipe_name          200 non-null    object 
 11  healthiness          200 non-null    object 
 12  Nutri_score          200 non-null    object 
 13  fsa_score            200 non-null    int64  
 14  SValue_Nor           200 non-null    float64
 15  Slabel               200 non-null    obj

In [8]:
## User Dataset concatenation based on user id (person)

# change the user_id to avoid intersection of different person
boosting_heal_df[['person']] = boosting_heal_df[['person']] + nudge_pers_df[['person']].max() + 1000
preference_user_df[['person']] = preference_user_df[['person']] + boosting_heal_df[['person']].max() + 1000
sustainability_user_id[['person']] = sustainability_user_id[['person']] + preference_user_df[['person']].max() + 1000

In [9]:
print(nudge_pers_df.person.min(), nudge_pers_df.person.max())
print(boosting_heal_df.person.min(), boosting_heal_df.person.max())
print(sustainability_user_id.person.min(), sustainability_user_id.person.max())

4 2132
3148 8204
14275 16385


In [10]:
### concatenate all user data to construct user DataFrame
user_features_DF = pd.concat([nudge_pers_df, boosting_heal_df, sustainability_user_id], axis=0, ignore_index=True)
user_features_DF.to_csv('./Dataset/user_features_df.csv', index=False)

In [11]:
## Fill NAN features
user_features_DF.isna().sum()

person                0
age                   0
country               0
education             0
diet_restriction    243
diet_goal           243
cooking_exp         243
eating_habits       243
gender                0
recipe_name           0
dtype: int64

In [12]:
# features with nan values
user_features_DF[['diet_restriction','diet_goal','cooking_exp','eating_habits']] 

,diet_restriction,diet_goal,cooking_exp,eating_habits
0,No dietary restrictions,No goals,Low,Unhealthy
1,No dietary restrictions,"Eat less salt,Eat less sugar,Eat more fruit,Ea...",Medium,Neither_healthy_no_unhealthy
2,No dietary restrictions,Lose weight,Low,Neither_healthy_no_unhealthy
3,No dietary restrictions,No goals,High,Neither_healthy_no_unhealthy
4,No dietary restrictions,Eat more fruit,High,Healthy
...,...,...,...,...
738,['Lactose intolerance'],['Lose weight'],Medium,Very_unhealthy
739,['No dietary restrictions'],"['Eat less salt', ' Gain weight']",Medium,Unhealthy
740,"['Lactose intolerance', ' Vegetarian']","['No goals', ' Eat more fruit', ' Eat more pro...",High,Healthy
741,"['Gluten free', ' Lactose intolerance']",['No goals'],Low,Very_unhealthy


In [13]:
## Replace Na values
## assuming that all users will have neutral value of features with NA
user_features_DF.fillna(
    {
    'diet_restriction':'No dietary restrictions',
    'diet_goal'       :'No goals',
    'cooking_exp'     : 'Medium',
    'eating_habits'    :'Neither_healthy_no_unhealthy', 
    }
, inplace=True)

In [14]:
user_features_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743 entries, 0 to 742
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   person            743 non-null    int64 
 1   age               743 non-null    object
 2   country           743 non-null    object
 3   education         743 non-null    object
 4   diet_restriction  743 non-null    object
 5   diet_goal         743 non-null    object
 6   cooking_exp       743 non-null    object
 7   eating_habits     743 non-null    object
 8   gender            743 non-null    object
 9   recipe_name       743 non-null    object
dtypes: int64(1), object(9)
memory usage: 58.2+ KB


In [15]:
## add id to chosen recipe and user
user_features_DF['user_id'] = user_features_DF['person']
user_features_DF['recipe_id'] = pd.Categorical(user_features_DF['recipe_name']).codes
user_features_DF.head()

,person,age,country,education,diet_restriction,diet_goal,cooking_exp,eating_habits,gender,recipe_name,user_id,recipe_id
0,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128
1,5,b25_35,US,High_school,No dietary restrictions,"Eat less salt,Eat less sugar,Eat more fruit,Ea...",Medium,Neither_healthy_no_unhealthy,Male,Sarge's EZ Pulled Pork BBQ,5,141
2,6,b35_45,US,BA,No dietary restrictions,Lose weight,Low,Neither_healthy_no_unhealthy,Male,Bow Ties with Sausage' Tomatoes and Cream,6,28
3,7,b25_35,US,High_school,No dietary restrictions,No goals,High,Neither_healthy_no_unhealthy,Male,Scott Hibb's Amazing Whisky Grilled Baby Back ...,7,143
4,9,b25_35,US,BA,No dietary restrictions,Eat more fruit,High,Healthy,Female,Pizza Without the Red Sauce,9,121


- The target feature from the user data is the recipe chosen by the user, represented by the recipe name

In [16]:
# extract unique selected recipes by the user
target_recipe = pd.DataFrame(user_features_DF['recipe_name'].unique(), columns=['recipes'])
target_recipe

,recipes
0,Ray's Chicken
1,Sarge's EZ Pulled Pork BBQ
2,Bow Ties with Sausage' Tomatoes and Cream
3,Scott Hibb's Amazing Whisky Grilled Baby Back ...
4,Pizza Without the Red Sauce
...,...
184,Chicken In Basil Cream
185,Rosemary Lemon Grilled Chicken
186,Beef Bulgogi
187,Honey Garlic Ribs


- create a column that indicate if user interact with a recipe or not, 1 if the user choses the recipes, 0 if not 

In [17]:
users = user_features_DF['user_id'].unique()
recipes = user_features_DF['recipe_id'].unique()

# (all combinations of users and recipes)
user_recipe_df = pd.MultiIndex.from_product([users, recipes], names= ['user_id','recipe_id']).to_frame(index=False)

# Add chosen index
user_recipe_df['chosen'] = user_recipe_df.merge(user_recipe_df.assign(chosen=1), on=['user_id', 'recipe_id'], how='left')['chosen'].fillna(0).astype(int)

# Merge with  all user features
user_features_DF = user_features_DF.merge(user_recipe_df, on='user_id')

In [18]:
user_features_DF.head()

,person,age,country,education,diet_restriction,diet_goal,cooking_exp,eating_habits,gender,recipe_name,user_id,recipe_id_x,recipe_id_y,chosen
0,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,128,1
1,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,141,1
2,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,28,1
3,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,143,1
4,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,121,1


In [19]:
user_features_DF.drop(columns = ['recipe_id_y'], inplace=True)
user_features_DF.rename(columns = {'recipe_id_x':'recipe_id'}, inplace=True)

In [20]:
user_features_DF.head()

,person,age,country,education,diet_restriction,diet_goal,cooking_exp,eating_habits,gender,recipe_name,user_id,recipe_id,chosen
0,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,1
1,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,1
2,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,1
3,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,1
4,4,b35_45,US,BA,No dietary restrictions,No goals,Low,Unhealthy,Male,Ray's Chicken,4,128,1


In [21]:
user_features_DF.shape

(140427, 13)

In [22]:
# replace the age with averages
age_mapping = {
    'under_18':18,
    'b18_24': 21,
    'b25_35': 30,
    'b35_45': 40,
    'b45_55': 50,
    'bover_55': 60,
}
user_features_DF['age'] = user_features_DF['age'].map(age_mapping)

#### Recipe Features
Construct a training dataset that describes each recipe using various features. The original dataset, primarily sourced from Allrecipes.com, serves as the basis for our research.

In [23]:
## read original recipes
source_recipes = pd.read_csv('./Dataset/recipe_source.csv')
source_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   URL                 942 non-null    object 
 1   Name                942 non-null    object 
 2   fiber_g             942 non-null    float64
 3   sodium_g            942 non-null    float64
 4   carbohydrates_g     942 non-null    float64
 5   fat_g               942 non-null    float64
 6   protein_g           942 non-null    float64
 7   sugar_g             942 non-null    float64
 8   saturate_g          942 non-null    float64
 9   size_g              942 non-null    float64
 10  Servings            942 non-null    int64  
 11  calories_kCal       942 non-null    int64  
 12  category            942 non-null    object 
 13  image_link          942 non-null    object 
 14  fat_100g            942 non-null    float64
 15  fiber_100g          942 non-null    float64
 16  sugar_10

In [24]:
## extract features for selected recipes by users
recip_features_df  = pd.merge(source_recipes, target_recipe, right_on = 'recipes', left_on='Name')

# check if all selected recipes by users are in the orignal recipe dataset
target_recipe.recipes.isin(source_recipes.Name).any()


True

In [25]:
# drop useless columns
source_recipes.drop(columns = ['URL','image_link'], inplace=True)

In [26]:
# rename a column
source_recipes.rename(columns={'Name':'recipe_name'}, inplace=True)

In [27]:
## final recipe dataset
source_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   recipe_name         942 non-null    object 
 1   fiber_g             942 non-null    float64
 2   sodium_g            942 non-null    float64
 3   carbohydrates_g     942 non-null    float64
 4   fat_g               942 non-null    float64
 5   protein_g           942 non-null    float64
 6   sugar_g             942 non-null    float64
 7   saturate_g          942 non-null    float64
 8   size_g              942 non-null    float64
 9   Servings            942 non-null    int64  
 10  calories_kCal       942 non-null    int64  
 11  category            942 non-null    object 
 12  fat_100g            942 non-null    float64
 13  fiber_100g          942 non-null    float64
 14  sugar_100g          942 non-null    float64
 15  saturated_100g      942 non-null    float64
 16  protien_

In [28]:
## add recipe add the to data using merge
source_recipes_df = source_recipes.merge(user_features_DF[['recipe_name', 'recipe_id']], on='recipe_name')

In [29]:
source_recipes_df.shape

(138726, 30)

## Two Tower Model
A Two-Tower Recommender System is a hybrid recommender model that integrates collaborative filtering and content-based methods. It utilizes two deep neural networks—one for users and one for items—to embed them into a shared low-dimensional space. The system then predicts user preferences or ratings for items based on the geometric relationships between these embeddings, using metrics such as cosine similarity or dot product. By leveraging both user-item interactions and the intrinsic features of users and items, the two-tower approach provides more accurate and personalized recommendations.
Figure below shows the two tower 



<p align="center">
  <img src="final_project.png" alt="Two-Tower Model" width="450"/>
</p>

The two-tower model will be implemented Pytorch

In [30]:
## Import libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from typing import List

In [31]:
## device used 
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" # apple hardware acceleration
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'using {device} device')

using mps device


In [32]:
## Concatenate user data and recipe data into one dataframe 
user_recipe_DF = user_features_DF.merge(source_recipes_df, on='recipe_id')
print(f'user data shape {user_features_DF.shape}, recipe data shape {source_recipes_df.shape}, all data combined shape {user_recipe_DF.shape}')

KeyboardInterrupt: 

In [ ]:
## train test split
train_data, test_data = train_test_split(user_recipe_DF, test_size=0.2)

In [ ]:
user_recipe_DF.columns

Index(['person', 'age', 'country', 'education', 'diet_restriction',
       'diet_goal', 'cooking_exp', 'eating_habits', 'gender', 'recipe_name_x',
       'user_id', 'recipe_id', 'chosen', 'recipe_name_y', 'fiber_g',
       'sodium_g', 'carbohydrates_g', 'fat_g', 'protein_g', 'sugar_g',
       'saturate_g', 'size_g', 'Servings', 'calories_kCal', 'category',
       'fat_100g', 'fiber_100g', 'sugar_100g', 'saturated_100g',
       'protien_100g', 'sodium_100mg', 'carbohydrates_100g', 'kj_100g',
       'Nutri_score', 'Fsa_new', 'salt_100g', 'salt_g', 'fat_count',
       'satfat_count', 'sugar_count', 'salt_count', 'NumberRatings'],
      dtype='object')

In [ ]:
# user_recipe_DF.drop(columns = ['recipe_name_x'], inplace=True)
# user_recipe_DF.rename(columns = {'recipe_name_y':'recipe_name'}, inplace=True)
user_recipe_DF.drop(columns = ['person'], inplace=True)

- Convert categorical Data into numerical

In [ ]:
from sklearn.preprocessing import LabelEncoder

# List of categorical features
categorical_columns = [
     'country', 'education', 'diet_restriction', 'diet_goal', 
    'cooking_exp', 'eating_habits', 'gender', 'recipe_name', 
     'category', 'Nutri_score'
]

# Choose whether to use Label Encoding or One-Hot Encoding
use_label_encoding = True  # Set to False for One-Hot Encoding

 # Choose whether to use Label Encoding or One-Hot Encoding
use_label_encoding = True  # Set to False for One-Hot Encoding

# Initialize LabelEncoders if using Label Encoding
label_encoders = {}
# Ensure categorical columns are treated as strings
for col in categorical_columns:
    user_recipe_DF[col] = user_recipe_DF[col].astype(str)

# Initialize LabelEncoders
label_encoders = {}

# Apply Label Encoding
for col in categorical_columns:
    le = LabelEncoder()
    user_recipe_DF[col] = le.fit_transform(user_recipe_DF[col])
    label_encoders[col] = le  # Save the encoder for later use


In [ ]:
# Costume the dataset and produce a tensor out of it

class CustomDataset(Dataset):
    def __init__(self, df:pd.DataFrame, user_features:List, item_features:List, label:List):
        self.user_id = df[user_features].values.astype(int)
        self.recipe_id = df[user_features].values.astype(int)
        self.chosen = df[label].values.astype(int)
        
    def __len__(self):
        return len(self.user_id)
    
    def __getitem__(self, idx):
        
        user_id  = self.user_id[idx]
        recipe_id = self.recipe_id[idx] 
        chosen   = self.chosen[idx]
        
        return [torch.tensor(user_id, dtype=torch.long), 
                torch.tensor(recipe_id, dtype=torch.long)], \
               torch.tensor(chosen, dtype=torch.float32)

In [ ]:
## Define user and item feature using DataLoeader

user_features = ['user_id', 'age','gender', 'country', 'education', 'diet_restriction',
       'diet_goal', 'cooking_exp', 'eating_habits'
       ]

recipe_features = ['recipe_id','recipe_name', 'fiber_g', 'sodium_g', 'carbohydrates_g', 'fat_g',
       'protein_g', 'sugar_g', 'saturate_g', 'size_g', 'Servings',
       'calories_kCal', 'category', 'fat_100g', 'fiber_100g', 'sugar_100g',
       'saturated_100g', 'protien_100g', 'sodium_100mg', 'carbohydrates_100g',
       'kj_100g', 'Nutri_score', 'Fsa_new', 'salt_100g', 'salt_g', 'fat_count',
       'satfat_count', 'sugar_count', 'salt_count', 'NumberRatings',
       ]

label = ['chosen']

train_dataloader = DataLoader(
    CustomDataset(
        train_data,
        user_features,
        item_features,
        label
    ), batch_size=64, shuffle=True
)

test_dataloader = DataLoader(
    CustomDataset(
        test_data,
        user_features,
        recipe_features,
        label
    ), batch_size=64, shuffle=True
    
)


ValueError: invalid literal for int() with base 10: 'b25_35'

In [ ]:
## the model configuration stored in the json
import json

with open('config.json','r') as f:
    conf = json.load(f)

In [ ]:
conf["user_embedding_num"]= 743+1
conf["item_embedding_num"]= 743+1
conf["user_embedding_dim"]= 9
conf["item_embedding_dim"]= 30
conf

{'user_embedding_num': 744,
 'user_embedding_dim': 9,
 'item_embedding_num': 744,
 'item_embedding_dim': 30,
 'user_dense': [50, 60, 20],
 'item_dense': [50, 60, 20],
 'activation': 'relu',
 'user_features': ['user_id',
  'age',
  'gender',
  'country',
  'education',
  'diet_restriction',
  'diet_goal',
  'cooking_exp',
  'eating_habits'],
 'item_features': ['recipe_id',
  'recipe_name',
  'fiber_g',
  'sodium_g',
  'carbohydrates_g',
  'fat_g',
  'protein_g',
  'sugar_g',
  'saturate_g',
  'size_g',
  'Servings',
  'calories_kCal',
  'category',
  'fat_100g',
  'fiber_100g',
  'sugar_100g',
  'saturated_100g',
  'protien_100g',
  'sodium_100mg',
  'carbohydrates_100g',
  'kj_100g',
  'Nutri_score',
  'Fsa_new',
  'salt_100g',
  'salt_g',
  'fat_count',
  'satfat_count',
  'sugar_count',
  'salt_count',
  'NumberRatings'],
 'label': ['chosen']}

In [293]:
## Creating two Tower Model

class TwoTower_rec(nn.Module):
    def __init__(self, conf):
        super(TwoTower_rec, self).__init__()
        self.user_embedding_num = conf["user_embedding_num"]
        self.user_embedding_dim = conf["user_embedding_dim"]
        
        self.item_embedding_num = conf["item_embedding_num"]
        self.item_embedding_dim = conf["item_embedding_dim"]
        self.user_embedding_dim2 = 200
        self.item_embedding_dim2 = 200
        
        
        self.user_dense = [len(user_features)*self.user_embedding_dim ,*conf["user_dense"]]
        self.item_dense = [len(item_features)*self.item_embedding_dim ,*conf["item_dense"]]
        self.activation = conf["activation"]
        self.model2 = False
        
        user_dense_layers = []
        item_dense_layers = []
        self.flatten = nn.Flatten()
        self.user_embedding = nn.Embedding(self.user_embedding_num,self.user_embedding_dim)
        for i  in range(len(self.user_dense) - 1):
            dense = nn.Linear(self.user_dense[i],self.user_dense[i+1])
            if self.activation == "relu":
                act = nn.ReLU()

            user_dense_layers.append(dense)
            user_dense_layers.append(act)

        self.user_tower = nn.Sequential(*user_dense_layers)
        
        self.item_embedding = nn.Embedding(self.item_embedding_num,self.item_embedding_dim)
        self.item_embedding = nn.Embedding(self.item_embedding_num,self.item_embedding_dim)
        for i  in range(len(self.item_dense) - 1):
            dense = nn.Linear(self.item_dense[i],self.item_dense[i+1])
            if self.activation == "relu":
                act = nn.ReLU()

            item_dense_layers.append(dense)
            item_dense_layers.append(act)
        
        self.item_tower = nn.Sequential(*item_dense_layers)
        if self.model2:
            self.user_embed2 = nn.Embedding(int(self.user_dense[-1]),self.user_embedding_dim2)
            self.item_embed2 = nn.Embedding(int(self.item_dense[-1]),self.item_embedding_dim2)
        
    def forward(self,X):
        user_embed = self.user_embedding(X[0])
        user_embed = self.flatten(user_embed)
        item_embed = self.item_embedding(X[1])
        item_embed = self.flatten(item_embed)

        user  = self.user_tower(user_embed)
        item  = self.item_tower(item_embed)
        if self.model2:
            user = self.user_embed2(user.long())
            item = self.item_embed2(item.long())
        score = torch.dot(user.reshape((-1,)), item.reshape((-1,)))
        return score
        

In [294]:
model = TwoTower_rec(conf)
model

TwoTower_rec(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (user_embedding): Embedding(744, 9)
  (user_tower): Sequential(
    (0): Linear(in_features=81, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=20, bias=True)
    (5): ReLU()
  )
  (item_embedding): Embedding(744, 30)
  (item_tower): Sequential(
    (0): Linear(in_features=900, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=20, bias=True)
    (5): ReLU()
  )
)

In [295]:
## Define the loww funcion and optimizer 
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [296]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = (X[0].to(device), X[1].to(device)), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y.float())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [297]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = (X[0].to(device), X[1].to(device)), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.float()).item()
    test_loss /= num_batches
    print(f"Test Error: \n , Avg loss: {test_loss:>8f} \n")

In [298]:
def fit(model, loss_fn, optimizer, train_dataloader, test_dataloader, epochs=5):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

In [299]:
fit(model, loss_fn, optimizer, train_dataloader, test_dataloader,)

Epoch 1
-------------------------------
1028764


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.